
## Orders Master Table – Implementation

The **Orders Master Table** serves as the central dataset for downstream analysis and modeling.

* Ensured that all upstream mart tables (**order items–products, reviews, payments, customers**) were aligned to the **order_id grain**.

* Validated grain consistency across all marts to guarantee a **single record per order**.

* Combined all order-level marts using **order_id as the join key**, preventing data duplication and maintaining referential integrity.

* Produced a unified, order-level master table containing behavioral, product, payment, review, customer, and geographic features.



In [104]:
import pandas as pd
pd.set_option("display.max_columns", None)

orders=pd.read_csv("../Source Data/olist_orders_dataset.csv")
prd_customers=pd.read_csv("../Processed Data/prd_customers.csv")
prd_products_orderitems=pd.read_csv("../Processed Data/prd_Products_OrderItems.csv")
prd_reviews=pd.read_csv("../Processed Data/prd_reviews.csv")
prd_payments=pd.read_csv("../Processed Data/prd_payments.csv")


## CHECK IF THE GRAIN IS UNIQUE BEFORE MERGE ##

In [96]:
orders['order_id'].duplicated().sum()
orders['customer_id'].duplicated().sum()
prd_customers['customer_id'].duplicated().sum()
prd_reviews['order_id'].duplicated().sum()
prd_payments['order_id'].duplicated().sum()
prd_products_orderitems['order_id'].duplicated().sum()

0

In [105]:
orders = orders.merge(
    prd_customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left"
)
orders = orders[~orders["order_status"].isin(["canceled", "unavailable"])]
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders = orders.sort_values(
    ["customer_unique_id", "order_purchase_timestamp", "order_id"]
).reset_index(drop=True)



In [107]:
orders["snapshot_date"] = orders["order_purchase_timestamp"] - pd.Timedelta(days=1)


In [108]:
orders["past_order_count"] = (
    orders.groupby("customer_unique_id").cumcount()
)
orders["is_repeat_customer"] = (orders["past_order_count"] > 0).astype(int)


In [109]:
orders["prev_purchase_ts"] = (
    orders.groupby("customer_unique_id")["order_purchase_timestamp"].shift(1)
)

orders["days_since_last_order"] = (
    (orders["order_purchase_timestamp"] - orders["prev_purchase_ts"])
    .dt.days
)
orders["is_first_order"] = orders["prev_purchase_ts"].isna().astype(int)
orders["days_since_last_order"] = orders["days_since_last_order"].fillna(9999).astype(int)


In [113]:
order_snapshots = orders[[
    "customer_id",
    "order_id",
    "order_purchase_timestamp",
    "snapshot_date",
    "is_repeat_customer",
    "past_order_count",
    "days_since_last_order",
    "is_first_order",
]].copy()


In [114]:
orders_final = (
    order_snapshots
    .merge(prd_customers, on="customer_id", how="left")
    .merge(prd_products_orderitems, on="order_id", how="left")
    .merge(prd_reviews, on="order_id", how="left")
    .merge(prd_payments, on="order_id", how="left")
)


## VALIDATION OF MASTER TABLE ##

In [117]:
orders_final.loc[orders_final["customer_unique_id"] == "fe81bb32c243a86b2f86fbf053fe6140"] \
      .sort_values("order_purchase_timestamp")

,customer_id,order_id,order_purchase_timestamp,snapshot_date,is_repeat_customer,past_order_count,days_since_last_order,is_first_order,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,tot_customers_in_zip,tot_sellers_in_zip,geolocation_lat,geolocation_lng,is_latlng_missing,n_distinct_categories,main_category,avg_photos_per_product,avg_desc_length,avg_weight,avg_volume,max_weight,max_volume,total_price,min_price,max_price,avg_price,total_freight_value,avg_freight_value,total_order_value,avg_order_value,min_order_value,max_order_value,total_items,min_items,max_items,review_count,avg_review_score,avg_review_length,has_review_text,min_review_score,max_review_score,pymt_mode_boleto,pymt_mode_credit_card,pymt_mode_debit_card,pymt_mode_not_defined,pymt_mode_voucher,avg_payment_installments,total_payment_installments,total_payment_value
97639,5e6e0e06c988f485293a7341c8e21871,f3e4f61152aed686fbbb83e794cc246d,2017-10-22 11:29:22,2017-10-21 11:29:22,0,0,9999,1,fe81bb32c243a86b2f86fbf053fe6140,1453,sao paulo,SP,10.0,0.0,-23.585069,-46.686721,0,1.0,watches_gifts,1.0,504.0,300.0,640.0,300.0,640.0,56.0,56.0,56.0,56.0,7.78,7.78,63.78,63.78,63.78,63.78,1.0,1.0,1.0,1.0,5.0,NaN,0.0,5.0,5.0,0.00,63.78,0.0,0.0,0.0,1.0,1.0,63.78
97640,a7c6653654e36720ded54cb3f92432a3,4b3c7f8318b056586e0407857f44a136,2017-10-31 16:20:25,2017-10-30 16:20:25,1,1,9,0,fe81bb32c243a86b2f86fbf053fe6140,1453,sao paulo,SP,10.0,0.0,-23.585069,-46.686721,0,1.0,watches_gifts,1.0,504.0,300.0,640.0,300.0,640.0,56.0,56.0,56.0,56.0,7.78,7.78,63.78,63.78,63.78,63.78,1.0,1.0,1.0,1.0,1.0,44.0,1.0,1.0,1.0,63.78,0.00,0.0,0.0,0.0,1.0,1.0,63.78
97641,2ba770a39f5caf13266b19d5095c0712,556b30ae47829a82927b5dff9ca2ba65,2018-02-23 14:34:34,2018-02-22 14:34:34,1,2,114,0,fe81bb32c243a86b2f86fbf053fe6140,1453,sao paulo,SP,10.0,0.0,-23.585069,-46.686721,0,1.0,cool_stuff,3.0,860.0,400.0,1920.0,400.0,1920.0,71.1,71.1,71.1,71.1,7.78,7.78,78.88,78.88,78.88,78.88,1.0,1.0,1.0,1.0,5.0,NaN,0.0,5.0,5.0,0.00,78.88,0.0,0.0,0.0,1.0,1.0,78.88
97642,fb827db7d93060684875e22f58033041,882af1e2ad3e4b132d4b23c429843dc8,2018-06-14 11:35:32,2018-06-13 11:35:32,1,3,110,0,fe81bb32c243a86b2f86fbf053fe6140,1453,sao paulo,SP,10.0,0.0,-23.585069,-46.686721,0,1.0,cool_stuff,2.0,712.0,5288.0,37440.0,5288.0,37440.0,1013.4,1013.4,1013.4,1013.4,22.39,22.39,1035.79,1035.79,1035.79,1035.79,1.0,1.0,1.0,1.0,5.0,46.0,1.0,5.0,5.0,0.00,1035.79,0.0,0.0,0.0,5.0,5.0,1035.79
97643,2f7e0d9b02c05f440fea44dd6d81e8b4,6baca7e224a54a0323eb003e011c9b8a,2018-06-21 12:10:25,2018-06-20 12:10:25,1,4,7,0,fe81bb32c243a86b2f86fbf053fe6140,1453,sao paulo,SP,10.0,0.0,-23.585069,-46.686721,0,1.0,watches_gifts,2.0,593.0,400.0,2880.0,400.0,2880.0,338.9,338.9,338.9,338.9,9.63,9.63,348.53,348.53,348.53,348.53,1.0,1.0,1.0,1.0,5.0,35.0,1.0,5.0,5.0,0.00,348.53,0.0,0.0,0.0,4.0,4.0,348.53


In [118]:
orders.loc[orders["customer_unique_id"] == "fe81bb32c243a86b2f86fbf053fe6140"] \
      .sort_values("order_purchase_timestamp")


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,snapshot_date,past_order_count,is_repeat_customer,prev_purchase_ts,days_since_last_order,is_first_order
97639,f3e4f61152aed686fbbb83e794cc246d,5e6e0e06c988f485293a7341c8e21871,delivered,2017-10-22 11:29:22,2017-10-22 11:45:21,2017-10-23 17:43:18,2017-10-24 20:34:01,2017-11-03 00:00:00,fe81bb32c243a86b2f86fbf053fe6140,2017-10-21 11:29:22,0,0,NaT,9999,1
97640,4b3c7f8318b056586e0407857f44a136,a7c6653654e36720ded54cb3f92432a3,delivered,2017-10-31 16:20:25,2017-11-01 03:26:13,2017-11-01 18:25:27,2017-11-03 20:27:30,2017-11-08 00:00:00,fe81bb32c243a86b2f86fbf053fe6140,2017-10-30 16:20:25,1,1,2017-10-22 11:29:22,9,0
97641,556b30ae47829a82927b5dff9ca2ba65,2ba770a39f5caf13266b19d5095c0712,delivered,2018-02-23 14:34:34,2018-02-23 14:51:07,2018-02-28 19:33:32,2018-03-01 21:31:22,2018-03-09 00:00:00,fe81bb32c243a86b2f86fbf053fe6140,2018-02-22 14:34:34,2,1,2017-10-31 16:20:25,114,0
97642,882af1e2ad3e4b132d4b23c429843dc8,fb827db7d93060684875e22f58033041,delivered,2018-06-14 11:35:32,2018-06-14 11:59:47,2018-06-15 14:08:00,2018-06-18 15:46:30,2018-06-29 00:00:00,fe81bb32c243a86b2f86fbf053fe6140,2018-06-13 11:35:32,3,1,2018-02-23 14:34:34,110,0
97643,6baca7e224a54a0323eb003e011c9b8a,2f7e0d9b02c05f440fea44dd6d81e8b4,delivered,2018-06-21 12:10:25,2018-06-21 12:40:49,2018-06-22 19:04:00,2018-06-25 17:50:47,2018-07-04 00:00:00,fe81bb32c243a86b2f86fbf053fe6140,2018-06-20 12:10:25,4,1,2018-06-14 11:35:32,7,0


## TOTAL ITEMS ##

In [119]:
orders_final.loc[orders_final["order_id"] =='8272b63d03f5f79c56e9e4120aec44ef']

,customer_id,order_id,order_purchase_timestamp,snapshot_date,is_repeat_customer,past_order_count,days_since_last_order,is_first_order,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,tot_customers_in_zip,tot_sellers_in_zip,geolocation_lat,geolocation_lng,is_latlng_missing,n_distinct_categories,main_category,avg_photos_per_product,avg_desc_length,avg_weight,avg_volume,max_weight,max_volume,total_price,min_price,max_price,avg_price,total_freight_value,avg_freight_value,total_order_value,avg_order_value,min_order_value,max_order_value,total_items,min_items,max_items,review_count,avg_review_score,avg_review_length,has_review_text,min_review_score,max_review_score,pymt_mode_boleto,pymt_mode_credit_card,pymt_mode_debit_card,pymt_mode_not_defined,pymt_mode_voucher,avg_payment_installments,total_payment_installments,total_payment_value
26611,fc3d1daec319d62d49bfb5e1f83123e9,8272b63d03f5f79c56e9e4120aec44ef,2017-07-16 18:19:25,2017-07-15 18:19:25,0,0,9999,1,4546caea018ad8c692964e3382debd19,5882,sao paulo,SP,11.0,0.0,-23.684604,-46.783471,0,1.0,health_beauty,2.666667,205.0,866.666667,1440.0,1000.0,1800.0,31.8,1.2,7.8,3.4,164.37,7.45,196.17,65.39,14.37,90.9,21.0,1.0,10.0,1.0,1.0,75.0,1.0,1.0,1.0,0.0,196.11,0.0,0.0,0.0,2.0,2.0,196.11


In [122]:
orders_final.to_csv("../Processed Data/prd_master_orders_customers.csv", index=False)